# Import python modules

In [ ]:
## import package bonanza
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy
import json
from pivottablejs import pivot_ui
import nltk
import unicodedata
import twitter_text
import re
from collections import Counter
import regex
from nltk.tokenize import RegexpTokenizer
from string import digits
from collections import defaultdict
import time
from configparser import ConfigParser
from gensim import corpora, models, similarities
tokenizer = RegexpTokenizer(r'\w+')
from string import digits
from geopy.geocoders import Nominatim, GoogleV3
import numpy as np
import math


# Set up sentiment dictionary

In [ ]:
# intitialize AFINN dictionary
afinn = dict(map(lambda (k,v): (k,int(v)), 
                     [ line.split('\t') for line in open("/Users/kristinaac/Desktop/insight_weather_project/AFINN/AFINN-111.txt") ]))



# Intialize postgres server

In [ ]:
#In Python: Define a database name 
# final_db, and your username for your computer (CHANGE IT BELOW). 
dbname = ''
username = ''

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url


## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

# Functions to convert temperatures and classify weather ratings

In [ ]:
# Devising a strategy to rate weather severity based on weather report 
#gathered from tweet location

# converts kelvin to F
def kelvin_to_F(t):
    return (t - 273.15)* 1.8000 + 32.00


# This is the master flat file with all the tweet related data
tweet_file = "/Users/kristinaac/Desktop/insight_weather_project/tweets/160128_full_tweet_data.txt"

# create a sorted list of all weather codes present in the dataset
weather_code_list = []
with open (tweet_file) as fh:
    for line in fh:
        weather_code = line.strip().split("\t")[4]
        #print weather_code
        weather_code_list.append(weather_code)
        line = eval(line.strip().split("\t")[-1])
        
        
weather_code_list = set(weather_code_list)

# 26 catagories of weather; I rated them from 1 to 8, with 8 being most severe. 
weather_dict = {'741':['Fog','3'],
'620': ['light_shower_snow','5'],
'601':['snow','6'],
'600':['light_snow','5'],
'602': ['heavy_snow','7'],
'300': ['light_intensity_drizzle','4'],
'301':['drizzle','4'],
'211': ['thunderstorm','7'],
'701': ['mist','2'],
'501': ['moderate_rain','6'],
'321': ['shower_drizzle','5'],
'520' : ['light_intensity_shower_rain','5'],
'521': ['shower_rain','6'],
'721': ['haze','2'],
'803': ['broken_clouds','2'],
'802': ['scattered_clouds','2'],
'801': ['few_clouds','2'],
'800': ['sky_is_clear','1'],
'502': ['heavy_intensity_rain','7'],
'500': ['light_rain','5'],
'804':['overcast_clouds','3'],
'761':['dust','3'],
'615':['light_rain_and_snow','5'],
'302':['heavy_intensity_drizzle','5'],
'202':['thunderstorm_with_heavy_rain','8'],
'310':['light_intensity_drizzle_rain','4']}

# Functions to process bounding box coordinates

In [ ]:
# convert bounding box coordinates into lat,long coordinates

def get_coordinates(coordinates):
    coord3= coordinates[0][0][0]
    coord1= coordinates[0][0][1]
    coord2 = coordinates[0][1][1]
    coord4 = coordinates[0][2][0]
        
    coords = (coord1,coord2,coord3,coord4)       
        
    centerx,centery = (numpy.average(coords[:2]),numpy.average(coords[2:]))
    centerx = float(centerx)
    centery = float(centery)
    return(centerx, centery)

# weather rating goes from 1-5 1 being the coldest

def rate_temp(current_temp):
    if int(current_temp) <= 32:
        temp_rating = 5
    if 32 < int(current_temp) <= 45:
        temp_rating = 4
    if 45 < int(current_temp) <= 55:
        temp_rating = 3
    if 55 < int(current_temp) <= 65:
        temp_rating = 2
    if int(current_temp) > 65:
        temp_rating = 1
    return temp_rating
        

#  Stopword and weather word lists

In [ ]:
#curated stoplist words

stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via',
            'one','com','new','like','great','make','top','say','would','thanks','thank','going',
            'new','use','should','could','see','want',
            'while','know','at_user','url','URL','barometer','Sunnyvale','mph','soundcloud','spotify']

# cureated wather_term list 

weather_term_list = ['rain','raindrops','raining','snow','sun','sunny',
                     'cloud','cloudy','thunderstorm','thunderstorms',
                     'monsoon','thunder','blizzard','icey','warm',
                     'hot','cold','weather','showers','flood','stormy','fog',
                     'chilly','brr','winter','storm','summer','spring','fall',
                     'blizzard2016','Blizzard2106','clouds','snowing','icy','snowball',
                     'snowstorm','ice','snowed','snowflakes''jonasblizzard','JonasBlizzard',
                     'Jonasblizzard','snowman','rainbow','snowiest', 'winters',
                     'snowboarder','snowboard','snowboarding','snowglobe','snowy','thundersnow',
                     'rains','snowpocalypse','rainy','snowdays','snowday','snowpack',
                     'firstsnow','foggy']




# Begin processing tweets and generating feature lists

In [ ]:
# intiate emmpty lists that will populate the pd dataframe
org_tweet_list =[]
search_term_list = []
stripped_tweet_list =[]
coord_lat_list = []
coord_long_list = []
location_list = []
retweet_list = []
time_stamp_list = []
tweets_with_emoji = []
weather_type_list = []
weather_rating_list = []
temp_rating_list  = []
temp_list = []
AFFIN_list = []
state_list = []
time_list = []
weather_related_words_list  = []
num_weather_related_words_list = []
total_words_list = []

# read through the tweet file above and start parsing

with open (tweet_file) as fh:
    for line in fh:
        tweet = line.strip().split("\t")[0]
        
        #lower search term case and add to list
        search_term = line.strip().split("\t")[1].lower()
        
        search_term_list.append(search_term)
        
        # identify weather code and append rating to list
        weather_code = line.strip().split("\t")[4]
        weather_type = weather_dict[weather_code][0]
        weather_rating = weather_dict[weather_code][1]
        
        weather_type_list.append(weather_type)
        weather_rating_list.append(weather_rating)
        
        # identify temp and append rating to list 
        temp_line = eval(line.strip().split("\t")[-1])
        current_temp = kelvin_to_F(int(temp_line['temp']))
        temp_rating = rate_temp(current_temp)
        temp_rating_list.append(temp_rating)
        temp_list.append(current_temp) 
        
        # add original tweet to tweet list
        mydata = json.loads(tweet)
        tweet = mydata['text'].strip().encode('ascii', 'ignore')
        org_tweet_list.append(tweet)
        
        # Begin to parse tweet for NPLP 

        emoji_tweet = tweet#.encode('ascii', 'ignore')
        fixed_tweet = tweet.encode('ascii', 'ignore').lower()
        fixed_tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',fixed_tweet)
        fixed_tweet = re.sub('@[^\s]+','AT_USER',fixed_tweet) 
        fixed_tweet = re.sub('[\s]+', ' ', fixed_tweet)
        fixed_tweet = re.sub(r'#([^\s]+)', r'\1', fixed_tweet)
        fixed_tweet = fixed_tweet.strip('\'"')
    
        # evaluated AFFIN score
        AFFIN_score = sum(map(lambda word: afinn.get(word, 0), fixed_tweet.lower().split()))
        AFFIN_list.append(AFFIN_score)
        
        fixed_tweet = tokenizer.tokenize(fixed_tweet.lower())
        # remove single and double letter words and stoplists
        unigrams = [w for w in fixed_tweet if len(w)==1]
        bigrams = [w for w in fixed_tweet if len(w)==2]
        stoplist  = set(nltk.corpus.stopwords.words("english") + stoplist_tw
                    + unigrams + bigrams)
        
        fixed_tweet = [token for token in fixed_tweet if token not in stoplist]
        # removes random digits
        fixed_tweet = [token for token in fixed_tweet if len(token.translate(None,digits)) == len(token)]

        #get count of weather related words, total words
        weather_related_words = [token for token in fixed_tweet if token in weather_term_list]
        num_weather_related_words = len(weather_related_words)
        
        if weather_related_words == []:
            num_weather_related_words = 0
            
        weather_related_words_list.append(weather_related_words)
        num_weather_related_words_list.append(num_weather_related_words)
        
        
        total_words = len([token for token in fixed_tweet])
        total_words_list.append(total_words)
        
        stripped_tweet_list.append(fixed_tweet)

        
        # get coordinates and append
        coordinates = mydata['place']['bounding_box']['coordinates']
        coord_1, coord_2, = get_coordinates(coordinates)
        coord_lat_list.append(coord_1)
        coord_long_list.append(coord_2)

        # get location and append
        location = mydata['place']['full_name'].encode('ascii', 'ignore').strip()
        finer_location = location.strip().split(',')
        finer_location = finer_location[len(finer_location)-1]
        location_list.append(location)
        state_list.append(finer_location)
        
        # get retweet_count
        retweet_count = mydata['retweet_count']
        retweet_list.append(retweet_count)
        
        # get time stamp 
        time_stamp = mydata['created_at'].encode('ascii', 'ignore').strip()
        fix_date = " ".join(time_stamp.strip().split(" ")[0:3])
        date = time_stamp.strip().split(" ")[2:4][0]
        clock = "".join(time_stamp.strip().split(" ")[2:4][1].split(":"))
        time = int("01%s%s" %(date,clock))
        time_stamp_list.append(fix_date)
        
        time_list.append(time)

        tweets_with_emoji.append(emoji_tweet)

# attempt get a count of emojis in each tweet

try:
    # Wide UCS-4 build
    e = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    e = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)
    
emoji_dict_list = []
for x in tweets_with_emoji: 
    emoji_list = []
    match  = e.search(x)
    if match:
        emoji_string = match.group()
        emoji_list = regex.findall(r'.\p{Sk}+|\X',emoji_string)
        #print emoji_list
        #emoji_dict = Counter(emoji_list)
    #emoji_dict_list.append(emoji_dict)

#print len(emoji_dict_list)




# Add list info to pandas dataframe

In [ ]:
# add it all into a a pd dataframe

tweet_data = pd.DataFrame()

tweet_data['date_stamp'] = [x for x in time_stamp_list]
tweet_data['time'] = [x for x in time_list]
tweet_data['location'] = [x for x in location_list]
tweet_data['state'] = [x for x in state_list]
tweet_data['lat'] = [x for x in coord_lat_list]
tweet_data['long'] = [x for x in coord_long_list]
tweet_data['org_text'] = [x for x in org_tweet_list]
tweet_data['fixed_text'] = [x for x in stripped_tweet_list]
tweet_data['retweet_count'] = [x for x in retweet_list]
tweet_data['term'] = [x for x in search_term_list]
tweet_data['temp'] = [x for x in temp_list]
tweet_data['weather'] = [x for x in weather_type_list]
tweet_data['weather_rating'] = [x for x in weather_rating_list]
tweet_data['temp_rating'] = [x for x in temp_rating_list]
tweet_data['affin_score'] = [x for x in AFFIN_list]
tweet_data['weather_words'] = [x for x in weather_related_words_list]
tweet_data['num_weather_words'] = [x for x in num_weather_related_words_list]
tweet_data['total_words'] = [x for x in total_words_list]

tweet_data['fixed_text'] = tweet_data['fixed_text'].apply(" ".join)
tweet_data['affin_score'] = tweet_data['affin_score'].apply(float)
tweet_data['weather_rating'] = tweet_data['weather_rating'].apply(float)
tweet_data['temp'] = tweet_data['temp'].apply(float)
tweet_data['retweet_count'] = tweet_data['retweet_count'].apply(float)
tweet_data['lat'] = tweet_data['lat'].apply(float)
tweet_data['long'] = tweet_data['long'].apply(float)
tweet_data['num_weather_words'] = tweet_data['num_weather_words'].apply(float)
tweet_data['total_words'] = tweet_data['total_words'].apply(float)
tweet_data['time'] = tweet_data['time'].apply(float)
tweet_data['date_stamp'] = tweet_data['date_stamp'].apply(str)

# filter out duplicates and non-weather related tweets

In [ ]:
# don't include the general tweets that I started collecting near the end
tweet_data = tweet_data.loc[tweet_data['term'] != 'general']
tweet_data = tweet_data.loc[tweet_data['num_weather_words'] > 0]
tweet_data['norm_response_score'] = (tweet_data['affin_score']/4)*tweet_data['weather_rating']

# Add dataframe to table in postgres

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
tweet_data.drop_duplicates(subset='org_text', keep='last').to_sql('tweet_weather_table', engine, if_exists='replace')

# Check to make sure the data can be accessed from the database

In [ ]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = ''
username = ''

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url



In [ ]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

#sql_query = """SELECT location,org_text,temp,weather_rating,norm_response_score, point(%s, %s) <@> point(long, lat)::point AS tweet_distance 
           #FROM tweet_weather_table WHERE (point(%s, %s) <@> point(long, lat)) < 50 
           #ORDER by tweet_distance;""" %(query_lon,query_lat,query_lon,query_lat)

sql_query = """SELECT * FROM tweet_weather_table;"""
tweet_data = pd.read_sql_query(sql_query,con)
tweet_data